In [1]:
import numpy as np
import pandas as pd
import time
import tkinter as tk
from openpyxl import load_workbook
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
servico = Service(ChromeDriverManager().install())
import funcoes as fc

In [55]:
#INICIA O NAVEGADOR
navegador = uc.Chrome(service=servico)
navegador.maximize_window()
navegador.get('https://sei.anatel.gov.br/')


In [56]:
#ENTRAR NA CAIXA DE PROCESSOS ATRIBUIDOS AO USUARIO
if fc.check_element_exists(By.XPATH,'//*[@id="divFiltro"]/div[2]/a', navegador):
    navegador.find_element(By.XPATH,'//*[@id="divFiltro"]/div[2]/a').click()
#CASO NAO ENCONTRE O FILTRO DE PROCESSOS ELE INFORMA QUE NAO ENTROU NO SEI
else:
    #APAGA O USUÁRIO
    navegador.find_element(By.XPATH,'//*[@id="txtUsuario"]').clear()
    print('Usuário ou senha incorretos. Digite novamente')

In [57]:
#PEGA O CORPO DA TABELA NO NAVEGADOR
tbody = navegador.find_element(By.XPATH, '//*[@id="tblProcessosRecebidos"]/tbody')
#PEGA TODAS AS LINHAS (TR) DO CORPO DA TABELA, EXCETO A PRIMEIRA
trs = tbody.find_elements(By.TAG_NAME, 'tr')[1:]

In [58]:
tr = trs[0]
print(tr.text)

53500.093241/2024-03 (lukasa.estagio)


In [59]:
def processa_tr(tr):
    #PEGA TODAS AS COLUNAS (TD) DA LINHA ATUAL (TR)
    tds = tr.find_elements(By.TAG_NAME, 'td')
    #INICIALIZA AS VARIÁVEIS QUE SERÃO USADAS PARA ARMAZENAR INFORMAÇÕES SOBRE O PROCESSO
    processo_texto = ""
    possui_anotacao = False
    aguardando_assinatura = False
    #ITERA SOBRE CADA COLUNA (TD) DA LINHA ATUAL (TR)
    for td in tds:
        #PEGA O TEXTO DA COLUNA ATUAL (TD)
        texto = td.text
        #VERIFICA SE O TEXTO NÃO CONTÉM A PALAVRA '.ESTAGIO'
        if '.estagio' not in texto:
            #REMOVE AS PALAVRAS 'RECEBIDOS' E ESPAÇOS EM BRANCO DO TEXTO
            texto = texto.replace('Recebidos', '').replace(' ', '')
            #ARMAZENA O TEXTO MODIFICADO NA VARIÁVEL 'PROCESSO_TEXTO'
            processo_texto = texto
        #PROCURA POR ÍCONES DE ANOTAÇÃO NA LINHA ATUAL (TR)
        anotacoes = tr.find_elements(By.XPATH, './/img[@src="svg/anotacao1.svg?18"]')
        #VERIFICA SE ENCONTROU ALGUM ÍCONE DE ANOTAÇÃO
        if anotacoes:
            #SE ENCONTROU, MARCA A VARIÁVEL 'POSSUI_ANOTACAO' COMO VERDADEIRA
            possui_anotacao = True
            #VERIFICA SE A ANOTAÇÃO CONTÉM O TEXTO 'AGUARDANDO ASSINATURA'
            if tr.find_elements(By.XPATH, ".//*[contains(translate(@onmouseover, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'aguardando assinatura')]"):
                #SE CONTÉM, MARCA A VARIÁVEL 'AGUARDANDO_ASSINATURA' COMO VERDADEIRA
                aguardando_assinatura = True
    #RETORNA AS INFORMAÇÕES DO PROCESSO COMO UMA TUPLA
    return (processo_texto.strip(), possui_anotacao, aguardando_assinatura)

In [60]:
lista_processos = []
lista_procConformes = []

for tr in trs:
    processo_texto, possui_anotacao, aguardando_assinatura = processa_tr(tr)
    if aguardando_assinatura:
        lista_procConformes.append(processo_texto)  
    elif not possui_anotacao:
        lista_processos.append(processo_texto) 

In [51]:
#INVERTE A ORDEM DOS PROCESSOS NAS LISTAS
lista_processos.reverse()

#IMPRIME A QUANTIDADE DE PROCESSOS QUE SERÃO ANALISADOS E CONCLUÍDOS
print("Quantidade de processos para analisar",len(lista_processos))
print("Quantidade de processos para concluir", len(lista_procConformes))

#COLETA CAMINHO DAS PLANILHAS DE EQUIPAMENTOS CONFORMES

planilhaDrones = f"C:\\Users\\lukasa.estagio\\ANATEL\\ORCN - Drones\\Lista de Drones Anatel_Corrigida.xlsx"
planilhaRadios = f"C:\\Users\\lukasa.estagio\\ANATEL\\ORCN - Rádios\\Lista Radiamador.xlsx"
planilhaGeral = f"C:\\Users\\lukasa.estagio\\ANATEL\\ORCN - DRONES SEI PLANILHA\\Distribuição Processo Drone.xlsx"


Quantidade de processos para analisar 27
Quantidade de processos para concluir 15


In [52]:
#FUNCAO QUE ESPERA UM ELEMENTO CARREGAR NA TELA E CLICA NELE
def clica_noelemento(navegador, modo_procura, element_id, tempo=10):
    # Espera até que o elemento seja carregado (exemplo: elemento localizado por ID)
    try:
        element = WebDriverWait(navegador, tempo).until(
            EC.element_to_be_clickable((modo_procura, element_id))
        )
        # Clica no elemento
        element.click()
        return True
    except TimeoutException:
        print(f"Elemento {element_id} não foi carregado no tempo esperado")
        return False

In [ ]:
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(lista_processos[0])
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
# #CRIA DESPACHO
# navegador.switch_to.frame('ifrConteudoVisualizacao')
# #time.sleep(2)
# #CLICA NO INCONE DE INCLUIR DOCUMENTO
# clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[1]')
# navegador.switch_to.frame('ifrVisualizacao')
# clica_noelemento(navegador, By.PARTIAL_LINK_TEXT,'Despacho Decisório')

In [63]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
#CLICA NO DESPACHO DECISORIO
clica_noelemento(navegador, By.PARTIAL_LINK_TEXT,"Despacho Decisório")
#navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#CLICA NO ICONE DE LEGO
clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[8]')

True

In [64]:
navegador.switch_to.frame('ifrVisualizacao')
select_element = navegador.find_element(By.ID, 'selBloco')
select = Select(select_element)
#PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
for opcao in select.options:
    if "Despachos para Drones aprovados" in opcao.text:
        select.select_by_visible_text(opcao.text)
        break
time.sleep(0.5)

In [68]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[19]')
navegador.switch_to.frame('ifrVisualizacao')
clica_noelemento(navegador, By.XPATH, '//*[@id="sbmSalvar"]')

True